In [1]:
from normality import normalize
import pandas as pd
from thefuzz import fuzz
from thefuzz import process
from countrynames import to_code
import os
import shutil
import datetime


In [2]:

csv_to_fix = "CrystalCompositionASTM_MVPVersion01_CI_20230605.csv"

now = datetime.datetime.now()
formatted_date = now.strftime("%Y%m%d")
folder_path = "%s_Country_Renamed_%s" %(csv_to_fix.split("_")[0], formatted_date)
os.makedirs(folder_path)
geo_df = pd.read_csv("./ISOToEcoInventToolSub/Geographies.csv")
df = pd.read_csv(csv_to_fix)



In [3]:
df['Country of Applicability'].unique()
df['Country of Applicability'] = "GLO"
df.to_csv("./%s/%s.csv"  %(folder_path, folder_path), index=False)

In [4]:
def normalize_name(country):
    """Clean up a country name before comparison."""
    return normalize(country, latinize=True)

df_ambig = pd.DataFrame(columns=["Input string", "Mapped name", "Mapped code", "Quarantine?"])

def mapping(country:str)->str:
    global df_ambig
    if(result:=to_code(country)) is not None:
        return result
    matching_rows = df_ambig[df_ambig["Input string"].str.contains(country, case=False, na=False)]
    if not matching_rows.empty:
        return country
    else:
        longname = process.extractOne(country, choices=geo_df['Name'], processor=(lambda x: str(x)), scorer=fuzz.token_set_ratio, score_cutoff=0)[0]
        shortname = geo_df[geo_df['Name'] == longname]['Shortname'].iloc[0]
        new_row = pd.DataFrame([{ 
            "Input string": country,
            "Mapped name": longname,
            "Mapped code": shortname}])
        df_ambig = pd.concat([df_ambig, new_row], ignore_index=True)
        return country
    
Region_column_name = 'Country of Applicability'
df[Region_column_name] = df[Region_column_name].apply(normalize_name)
df[Region_column_name] =df[Region_column_name].apply(lambda x: mapping(x))

df_ambig.to_csv("ambig.csv", index=False)
df_ambig.to_csv("./%s/AMBIG_%s.csv" %(folder_path, folder_path), index=False)


In [5]:
input("Go to ambig.csv and correct ambiguous inputs")

''

In [5]:
def split_ambig():
    ambig = pd.read_csv('./ambig.csv')
    if ambig.empty:
        return ambig, ambig
    corrected = ambig[ambig['Quarantine?'] != 'x']
    further_quarantined = ambig[ambig['Quarantine?'] == 'x']
    return corrected, further_quarantined

def remove_quarantined(df_first_clean, further_quarantined, corrected):
    overlap_values = further_quarantined['Input string'].unique()
    removed_rows_df = df_first_clean[df_first_clean['Country of Applicability'].isin(overlap_values)].copy()
    df_second_clean = df_first_clean[~df_first_clean['Country of Applicability'].isin(overlap_values)]
    
    merged_df = pd.merge(df_second_clean, corrected, left_on='Country of Applicability', right_on='Input string', how='left')
    merged_df.loc[merged_df['Input string'].notna(), 'Country of Applicability'] = merged_df['Mapped code']
    merged_df.drop(columns=['Mapped code', "Input string","Mapped name","Quarantine?"], inplace=True)
    
    file_path = "./ambig.csv"
    if os.path.exists(file_path):
        shutil.move(file_path, "./%s/ambig_VERIFIED_%s.csv" % (folder_path, folder_path))
    return merged_df, removed_rows_df



corrected, further_quarantined = split_ambig()
merged_df, removed_rows_df =remove_quarantined(df, further_quarantined, corrected)




In [6]:
# In this cell, please add any code you'd like to edit 
print(merged_df['Country of Applicability'].unique())
print(removed_rows_df['Country of Applicability'].unique())


['AW' 'AF' 'AFRICA' 'AO' 'AL' 'AE' 'AR' 'AM' 'AS' 'UN-ASIA/UN-OCEANIA'
 'AQ' 'AG' 'AU' 'AT' 'AZ' 'BI' 'BE' 'BJ' 'BF' 'BD' 'BG' 'BH' 'BS' 'BA'
 'BY' 'BZ' 'BM' 'BO' 'BB' 'BN' 'BT' 'BW' 'CF' 'CH' 'CL' 'CI' 'CM' 'CD'
 'CG' 'CK' 'CO' 'KM' 'CV' 'CR' 'UN-CAMERICA' 'CSHH' 'CU' 'KY' 'CY' 'CZ'
 'DE' 'DJ' 'DM' 'DK' 'DO' 'DZ' 'EC' 'EG' 'ER' 'EH' 'ES' 'EE' 'ET' 'EU'
 'RER/RAS' 'RER' 'FI' 'FJ' 'FK' 'FR' 'FO' 'FM' 'GA' 'GB' 'GE' 'GH' 'GI'
 'GN' 'GP' 'GM' 'GW' 'GQ' 'GR' 'GD' 'GL' 'GT' 'GF' 'GU' 'GY' 'HK' 'HN'
 'HR' 'HT' 'HU' 'ID' 'IE' 'IR' 'IQ' 'IS' 'IL' 'IT' 'JM' 'JO' 'KZ' 'KE'
 'KG' 'KH' 'KI' 'KN' 'KR' 'KW' 'LA' 'LB' 'LR' 'LY' 'LC' 'LK' 'LS' 'LT'
 'LU' 'LV' 'MO' 'MA' 'MD' 'MG' 'MV' 'MX' 'RME' 'MK' 'ML' 'MT' 'MM' 'ME'
 'MN' 'MP' 'MZ' 'MR' 'MS' 'MQ' 'MU' 'MW' 'MY' 'NA' 'NC' 'NE' 'NG' 'NI'
 'NU' 'NL' 'ANHH' 'RNA' ' non-OECD' 'NO' 'NP' 'NR' 'NZ' ' OECD-AS/OECD-OC'
 'oecd' 'OECD-EU' ' OECD-NA' 'OM' 'OPEC-AF' 'opec' ' non-OPEC' 'OPEC-SA'
 'PK' 'PA' 'PE' 'Persian Gulf' 'PH' 'PG' 'PL' 'PR' 'KP' 'PT' 'PY' 'P

In [7]:
input("Look through each dataset and adjust any inconsistent values")




In [8]:
merged_df.loc[merged_df['Country of Applicability'] == 'former yugoslavia'] = 'YUCS'
merged_df['Country of Applicability'].unique()

/var/folders/r3/p8cv621n1mvcj2jybcrwr3tm0000gr/T/ipykernel_30390/1155168044.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'YUCS' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  merged_df.loc[merged_df['Country of Applicability'] == 'former yugoslavia'] = 'YUCS'
/var/folders/r3/p8cv621n1mvcj2jybcrwr3tm0000gr/T/ipykernel_30390/1155168044.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'YUCS' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merged_df.loc[merged_df['Country of Applicability'] == 'former yugoslavia'] = 'YUCS'


array(['AW', 'AF', 'AFRICA', 'AO', 'AL', 'AE', 'AR', 'AM', 'AS',
       'UN-ASIA/UN-OCEANIA', 'AQ', 'AG', 'AU', 'AT', 'AZ', 'BI', 'BE',
       'BJ', 'BF', 'BD', 'BG', 'BH', 'BS', 'BA', 'BY', 'BZ', 'BM', 'BO',
       'BB', 'BN', 'BT', 'BW', 'CF', 'CH', 'CL', 'CI', 'CM', 'CD', 'CG',
       'CK', 'CO', 'KM', 'CV', 'CR', 'UN-CAMERICA', 'CSHH', 'CU', 'KY',
       'CY', 'CZ', 'DE', 'DJ', 'DM', 'DK', 'DO', 'DZ', 'EC', 'EG', 'ER',
       'EH', 'ES', 'EE', 'ET', 'EU', 'RER/RAS', 'RER', 'FI', 'FJ', 'FK',
       'FR', 'FO', 'FM', 'GA', 'GB', 'GE', 'GH', 'GI', 'GN', 'GP', 'GM',
       'GW', 'GQ', 'GR', 'GD', 'GL', 'GT', 'GF', 'GU', 'GY', 'HK', 'HN',
       'HR', 'HT', 'HU', 'ID', 'IE', 'IR', 'IQ', 'IS', 'IL', 'IT', 'JM',
       'JO', 'KZ', 'KE', 'KG', 'KH', 'KI', 'KN', 'KR', 'KW', 'LA', 'LB',
       'LR', 'LY', 'LC', 'LK', 'LS', 'LT', 'LU', 'LV', 'MO', 'MA', 'MD',
       'MG', 'MV', 'MX', 'RME', 'MK', 'ML', 'MT', 'MM', 'ME', 'MN', 'MP',
       'MZ', 'MR', 'MS', 'MQ', 'MU', 'MW', 'MY', 'NA', 'NC', 

In [9]:
removed_rows_df.to_csv("./%s/%s_further_quarantined.csv" %(folder_path, folder_path), index=False) 
merged_df.to_csv("./%s/%s.csv"  %(folder_path, folder_path), index=False)


In [4]:
shutil.copy("./Nationalizer.ipynb", "%s" % folder_path)
shutil.move(csv_to_fix, "%s" % folder_path)
shutil.make_archive(folder_path.split('.')[0], 'zip', folder_path)
shutil.rmtree(folder_path)
